In [ ]:
frfrom0om google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:
def train_cnn_model(train_data_dir, val_data_dir, epochs=5, batch_size=32):
    # Define data generators for training and validation data
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary'
    )

    val_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary'
    )

    # Create CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(train_generator, epochs=epochs, batch_size=batch_size, validation_data=val_generator)

    return model, history

def test_model(model, test_data_dir):
    # Define data generator for testing data
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(150, 150),
        batch_size=1,
        class_mode='binary',
        shuffle=False
    )

    # Evaluate the model on the testing data
    loss, accuracy = model.evaluate(test_generator)
    print("Test Accuracy:", accuracy)

    # # Display images and predicted labels
    # for i in range(len(test_generator)):
    #     image, label = test_generator[i]
    #     prediction = model.predict(image)
    #     predicted_label = "Real" if prediction > 0.5 else "Fake"
    #     true_label = "Real" if label > 0.5 else "Fake"

    #     plt.imshow(image[0])
    #     plt.title(f"Predicted: {predicted_label}, True: {true_label}")
    #     plt.axis('off')
    #     plt.show()

# Example usage
train_data_dir = '/content/drive/MyDrive/dataset_pls/train'  # Path to directory containing training images
val_data_dir = '/content/drive/MyDrive/dataset_pls/valid'  # Path to directory containing validation images
test_data_dir = '/content/drive/MyDrive/dataset_pls/test'    # Path to directory containing testing images

# Train the CNN model
trained_model, history = train_cnn_model(train_data_dir, val_data_dir)

# Test the trained model
test_model(trained_model, test_data_dir)


Found 3082 images belonging to 2 classes.
Found 1522 images belonging to 2 classes.
Epoch 1/5
97/97 [==============================] - 778s 8s/step - loss: 0.5421 - accuracy: 0.7310 - val_loss: 0.5441 - val_accuracy: 0.7490
Epoch 2/5
97/97 [==============================] - 56s 571ms/step - loss: 0.3828 - accuracy: 0.8433 - val_loss: 0.3916 - val_accuracy: 0.8292
Epoch 3/5
97/97 [==============================] - 55s 573ms/step - loss: 0.2936 - accuracy: 0.8816 - val_loss: 0.5381 - val_accuracy: 0.6859
Epoch 4/5
97/97 [==============================] - 56s 575ms/step - loss: 0.2515 - accuracy: 0.9010 - val_loss: 0.3302 - val_accuracy: 0.8719
Epoch 5/5
97/97 [==============================] - 55s 569ms/step - loss: 0.1829 - accuracy: 0.9299 - val_loss: 0.2912 - val_accuracy: 0.9034
Found 1467 images belonging to 2 classes.
1467/1467 [==============================] - 199s 136ms/step - loss: 0.3085 - accuracy: 0.9250
Test Accuracy: 0.9250170588493347


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Define data generator for testing data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

# Evaluate the model on the testing data
loss, accuracy = trained_model.evaluate(test_generator)
print("Test Accuracy:", accuracy)

# Get true labels
true_labels = test_generator.labels

# Predict labels for the test data
predictions = trained_model.predict(test_generator)
predicted_labels = (predictions > 0.5).astype(int)

# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
